# Stokes drift at ocean station Papa

This notebook computes the surface Stokes drift from the momentum fluxes assuming a constant Langmuir number.

In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd
sys.path.append(os.path.join(os.pardir, 'gotmtool'))
from gotmtool import *

### Load data

In [ ]:
paparoot = os.path.join(os.environ['HOME'], 'data', 'OCS', 'Papa', 'Papa_20100616-20150101')
startdate = '2012-03-20'
enddate = '2013-03-22'
La_turb = 0.31
rho = 1027.

### Compute Stokes drift

In [ ]:
# load momentum flux input data
fname = 'tau50n145w_hr.cdf'
ds = xr.load_dataset(os.path.join(paparoot, fname)).sel(time=slice(startdate, enddate))
if '_FillValue' in ds.attrs:
    skipvalue = ds.attrs['_FillValue']
elif 'missing_value' in ds.attrs:
    skipvalue = ds.attrs['missing_value']
else:
    skipvalue = None
time = ds.time.values
ntime = time.size
dttime = [pd.Timestamp(time[i]).to_pydatetime() for i in np.arange(ntime)]
# vector variable
vnamex = 'TX_442'
vnamey = 'TY_443'
datax = ds.data_vars[vnamex].values.squeeze()
datay = ds.data_vars[vnamey].values.squeeze()
us0 = np.sign(datax)*np.sqrt(np.abs(datax)/rho)*La_turb**(-2)
vs0 = np.sign(datay)*np.sqrt(np.abs(datay)/rho)*La_turb**(-2)
us0[datax==skipvalue] = np.nan
vs0[datay==skipvalue] = np.nan

### Save Stokes drift to file

In [ ]:
dat_dump_ts(dttime, [us0, vs0], 'ospapa_us0_La0p{:d}.dat'.format(int(La_turb*100))
            , skip_value=np.nan)